In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Importing API key
from api_keys import g_key


In [53]:
path_file = "../WeatherPy/output_data_file"
data_file = pd.read_csv(path_file)
data_file.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Jilin,100,CN,1583770247,91,43.00,126.00,24.69,3.42
1,Ushuaia,40,AR,1583770249,57,-54.80,-68.30,51.80,11.41
2,Hermanus,100,ZA,1583770250,85,-34.42,19.23,73.99,5.01
3,Amapá,90,BR,1583770251,54,1.00,-52.00,85.89,5.28
4,Ancud,20,CL,1583770252,72,-41.87,-73.82,59.00,3.36


In [54]:
#Configuring gmaps

gmaps.configure(api_key=g_key)

coordinates = data_file[["Latitude","Longitude"]]
humidity = data_file["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '950px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plotting Heatmap
fig = gmaps.figure(layout =figure_layout)

# Creating heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False,max_intensity=150,
                                 point_radius=3)


# Adding layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [55]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

vacation_file = data_file.loc[(data_file["Max Temperature"]<80) & (data_file["Max Temperature"]>70),:]
vacation_file = vacation_file.loc[(vacation_file["Wind Speed"]<10) & (vacation_file["Cloudiness"]==0),:]
vacation_file

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
41,Mātherān,0,IN,1583770296,69,18.98,73.27,78.80,5.82
43,Ahvāz,0,IR,1583770298,25,31.32,48.67,75.20,4.70
60,Tabuk,0,PH,1583770318,89,17.42,121.44,74.01,4.27
236,Pandan,0,PH,1583770540,80,14.05,124.17,78.69,6.93
269,Poya,0,NC,1583770578,73,-21.35,165.15,79.77,8.41
276,Pisco,0,PE,1583770586,30,-13.70,-76.22,79.00,6.93
277,Wanning,0,CN,1583770587,93,18.80,110.40,74.19,4.63
395,Cidreira,0,BR,1583770733,75,-30.18,-50.21,73.29,7.02
445,Tuyen Quang,0,VN,1583770794,48,21.82,105.22,76.71,2.04
484,Digha,0,IN,1583770841,63,21.68,87.55,78.69,6.76


In [56]:
# set up a parameters dictionary
params = {
    "radius": 5000,
    "type": "restaurant",
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
for index, row in vacation_file.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]
    params["location"] = f"{lat},{lng}"

    # run a request using params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    hotel_data = response.json()
    results = hotel_data["results"]

    # Print the json (pretty printed)
    try:
        print(f"The first hotel within 5000m in {city} is {results[0]['name']}.")
        
        vacation_file.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

The first hotel within 5000m in Mātherān is Hotel Gulshan Heritage.
------------
The first hotel within 5000m in Ahvāz is Sufi Burger.
------------
The first hotel within 5000m in Tabuk is Davidson Hotel.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
The first hotel within 5000m in Pisco is Hostal El Candelabro.
------------
The first hotel within 5000m in Wanning is 肯德基万宁餐厅.
------------
The first hotel within 5000m in Cidreira is Restaurante Andrei.
------------
The first hotel within 5000m in Tuyen Quang is Bear Tea Tuyên Quang.
------------
The first hotel within 5000m in Digha is fun 'n' food.
------------
The first hotel within 5000m in Balabac is Divine's Carenderia.
------------
The first hotel within 5000m in Bogale is Nade Garden.
------------


In [57]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

#Hotel Map
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

In [59]:
hotel_df = vacation_file.dropna(how="any")
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
41,Mātherān,0,IN,1583770296,69,18.98,73.27,78.80,5.82,Hotel Gulshan Heritage
43,Ahvāz,0,IR,1583770298,25,31.32,48.67,75.20,4.70,Sufi Burger
60,Tabuk,0,PH,1583770318,89,17.42,121.44,74.01,4.27,Davidson Hotel
276,Pisco,0,PE,1583770586,30,-13.70,-76.22,79.00,6.93,Hostal El Candelabro
277,Wanning,0,CN,1583770587,93,18.80,110.40,74.19,4.63,肯德基万宁餐厅
395,Cidreira,0,BR,1583770733,75,-30.18,-50.21,73.29,7.02,Restaurante Andrei
445,Tuyen Quang,0,VN,1583770794,48,21.82,105.22,76.71,2.04,Bear Tea Tuyên Quang
484,Digha,0,IN,1583770841,63,21.68,87.55,78.69,6.76,fun 'n' food
487,Balabac,0,PH,1583770845,81,7.99,117.06,79.41,4.03,Divine's Carenderia
497,Bogale,0,MM,1583770858,71,16.28,95.40,70.95,9.28,Nade Garden


In [60]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [62]:
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…